In [1]:
import gurobipy as gp
from gurobipy import GRB

# Max x1 + 2x2
# s.t -x1 - x2 <= -6
#     2x1 - x2 <= 6
#     -x1 + x2 <= 3
#     x1, x2 >= 0

model = gp.Model("HW3_BT1")

# Khai bao bien
x1 = model.addVar(vtype=GRB.CONTINUOUS, name="x1", lb=0)
x2 = model.addVar(vtype=GRB.CONTINUOUS, name="x2", lb=0)

# Ham muc tieu
model.setObjective(x1 + 2 * x2, GRB.MAXIMIZE)

# Them rang buoc

model.addConstr(-x1 - x2 <= -6, "c1")
model.addConstr(2 * x1 - x2 <= 6, "c2")
model.addConstr(-x1 + x2 <= 3, "c3")

# Giai bai toan
model.optimize()

# In ket qua
if model.status == GRB.OPTIMAL:
    print(f"Optimal objective value: {model.objVal}")
    print(f"Optimal values: x1 = {x1.x}, x2 = {x2.x}")
else:
    print("No optimal solution found.")

Set parameter Username
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 5 7535HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 3 rows, 2 columns and 6 nonzeros
Model fingerprint: 0xb8ccd630
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 6e+00]
Presolve removed 3 rows and 2 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.3000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  3.300000000e+01
Optimal objective value: 33.0
Optimal values: x1 = 9.0, x2 = 12.0


In [4]:
import numpy as np

# Hàm hiển thị bảng đơn hình tại mỗi bước
def print_tableau(tableau, basic_vars, non_basic_vars):
    print("Bảng đơn hình:")
    header = ["z"] + [f"x{i+1}" for i in non_basic_vars] + [f"s{i+1}" for i in range(len(basic_vars))] + ["b"]
    print(" | ".join(header))
    print("-" * (len(header) * 8))
    for i, row in enumerate(tableau):
        if i == 0:
            row_header = "z"
        else:
            row_header = f"s{basic_vars[i-1]+1}"
        print(f"{row_header} | " + " | ".join(f"{val:.2f}" for val in row))

# Hàm thực hiện thuật toán đơn hình dạng từ điển
def simplex(c, A, b):
    # Khởi tạo tableau
    num_constraints, num_variables = A.shape
    tableau = np.zeros((num_constraints + 1, num_variables + num_constraints + 1))

    # Thiết lập hàng hàm mục tiêu (hàng đầu tiên)
    tableau[0, 0:num_variables] = -c

    # Thiết lập ma trận ràng buộc
    tableau[1:, 0:num_variables] = A

    # Thêm ma trận đơn vị cho biến phụ
    tableau[1:, num_variables:num_variables + num_constraints] = np.eye(num_constraints)

    # Thêm cột hệ số b (hệ số hằng)
    tableau[1:, -1] = b

    # Biến phụ s1, s2, s3 là biến cơ sở ban đầu
    basic_vars = list(range(num_variables, num_variables + num_constraints))
    non_basic_vars = list(range(num_variables))

    print("Bảng đơn hình ban đầu:")
    print_tableau(tableau, basic_vars, non_basic_vars)

    # Lặp thuật toán đơn hình
    iteration = 0
    while True:
        iteration += 1
        print(f"\nIteration {iteration}")

        # Kiểm tra điều kiện tối ưu (tất cả các hệ số trong hàng z phải không âm)
        if all(tableau[0, :-1] >= 0):
            print("Đã đạt đến giải pháp tối ưu.")
            break

        # Chọn biến vào: cột có hệ số âm nhỏ nhất trong hàng đầu tiên (hàm mục tiêu)
        pivot_col = np.argmin(tableau[0, :-1])

        # Tìm biến ra: dựa trên tỷ số giữa cột hằng và cột pivot
        if all(tableau[1:, pivot_col] <= 0):
            raise Exception("Bài toán không có nghiệm hữu hạn")

        ratios = tableau[1:, -1] / tableau[1:, pivot_col]
        ratios[tableau[1:, pivot_col] <= 0] = np.inf  # Không xét tỷ số âm

        # Tìm hàng có tỷ số nhỏ nhất (biến ra)
        pivot_row = np.argmin(ratios) + 1

        # Pivot element
        pivot_val = tableau[pivot_row, pivot_col]

        print(f"Biến vào: x{pivot_col + 1}, Biến ra: s{basic_vars[pivot_row - 1] + 1}")

        # Chia hàng pivot cho giá trị pivot
        tableau[pivot_row, :] /= pivot_val

        # Thực hiện biến đổi Gauss cho các hàng khác
        for i in range(len(tableau)):
            if i != pivot_row:
                tableau[i, :] -= tableau[i, pivot_col] * tableau[pivot_row, :]

        # Cập nhật biến cơ sở và không cơ sở
        basic_vars[pivot_row - 1], non_basic_vars[pivot_col] = non_basic_vars[pivot_col], basic_vars[pivot_row - 1]

        # In bảng đơn hình sau khi biến đổi
        print_tableau(tableau, basic_vars, non_basic_vars)

    # Lấy giá trị tối ưu
    optimal_value = tableau[0, -1]
    
    # Lấy nghiệm cho các biến quyết định
    solution = np.zeros(num_variables)
    for i in range(1, num_constraints + 1):
        basic_var_index = np.where(tableau[i, 0:num_variables] == 1)[0]
        if len(basic_var_index) == 1:
            solution[basic_var_index[0]] = tableau[i, -1]
    
    return solution, optimal_value

# Hàm chính
def main():
    # Dữ liệu bài toán
    c = np.array([1, 2])  # Hệ số hàm mục tiêu
    A = np.array([[-1, -1], [2, -1], [-1, 1]])  # Ma trận ràng buộc
    b = np.array([-6, 6, 3])  # Hệ số hằng của ràng buộc

    # Giải bài toán
    solution, optimal_value = simplex(c, A, b)

    # Kết quả
    print("\nNghiệm tối ưu:", solution)
    print("Giá trị tối ưu của hàm mục tiêu:", optimal_value)

if __name__ == "__main__":
    main()


Bảng đơn hình ban đầu:
Bảng đơn hình:
z | x1 | x2 | s1 | s2 | s3 | b
--------------------------------------------------------
z | -1.00 | -2.00 | 0.00 | 0.00 | 0.00 | 0.00
s3 | -1.00 | -1.00 | 1.00 | 0.00 | 0.00 | -6.00
s4 | 2.00 | -1.00 | 0.00 | 1.00 | 0.00 | 6.00
s5 | -1.00 | 1.00 | 0.00 | 0.00 | 1.00 | 3.00

Iteration 1
Biến vào: x2, Biến ra: s5
Bảng đơn hình:
z | x1 | x5 | s1 | s2 | s3 | b
--------------------------------------------------------
z | -3.00 | 0.00 | 0.00 | 0.00 | 2.00 | 6.00
s3 | -2.00 | 0.00 | 1.00 | 0.00 | 1.00 | -3.00
s4 | 1.00 | 0.00 | 0.00 | 1.00 | 1.00 | 9.00
s2 | -1.00 | 1.00 | 0.00 | 0.00 | 1.00 | 3.00

Iteration 2
Biến vào: x1, Biến ra: s4
Bảng đơn hình:
z | x4 | x5 | s1 | s2 | s3 | b
--------------------------------------------------------
z | 0.00 | 0.00 | 0.00 | 3.00 | 5.00 | 33.00
s3 | 0.00 | 0.00 | 1.00 | 2.00 | 3.00 | 15.00
s1 | 1.00 | 0.00 | 0.00 | 1.00 | 1.00 | 9.00
s2 | 0.00 | 1.00 | 0.00 | 1.00 | 2.00 | 12.00

Iteration 3
Đã đạt đến giải pháp tối ư